In [ ]:
from features.feature_constructor import Feature, GEO_GRAIN_LEN_MAP
from features.violence_calls import ViolenceCalls
from features.population import Population
from detroit_geos import get_detroit_census_geos, get_detroit_boundaries
import geopandas as gpd

In [9]:
import numpy as np
import matplotlib.pyplot as plt

# Framework
`help(Feature)` gives a reasonable docstring, but may be easier to just look at the code, which contains descriptions of every method

We'll concentrate on using a child class - population

## Population example

In [ ]:
# The Population class contains reference information for the data set
# Instantiating the object does not load the data by default

In [ ]:
population = Population(decennial_census_year=2010)

In [ ]:
# loading the data is not a raw load, it contains transforms that we well always want, like column selection and renaming
population.load_data()
population.data.head()

# This decorated method standardizes block ids and does validation, in addition to any additional cleaning you may want to do
population.cleanse_data()

In [ ]:
population.data.head()

In [ ]:
# this decorated method will automatically run the above steps and load data + set the index as necessary if it hasn't be done yet
# It defaults to full data load, so if the data set is large, you may want to load the data with custom arguments instead of doing 
# it automatically

# Outputs a series, which is the goal of this class
block_group_population = population.construct_feature(target_geo_grain='block group')
block_group_population 

In [ ]:
block_group_population.hist(bins=np.arange(0,3000,100))
_ = plt.gca().set(title='looks like a hurdle binomial distribution',xlabel='population',ylabel='n block groups')

## Violent calls example
spoiler: it works the same, but with some extra args

In [ ]:
from features.feature_constructor import Feature
from features.violence_calls import ViolenceCalls
from features.population import Population
from detroit_geos import get_detroit_census_geos

In [ ]:
call_feat = ViolenceCalls(decennial_census_year=2010)
call_feat.load_data(use_lat_long=False,sample_rows=10000)
# # call_feat.cleanse_data()
# call_feat.construct_feature(target_geo_grain='block')

In [ ]:
call_feat.data.head()

In [ ]:
# without reloading or recleaning the data, you can regenerate a new feature vector on a different grain
ftr = call_feat.construct_feature(target_geo_grain='tract')

ftr

In [ ]:
call_feat.open_data_url('source')

# Rental Statuses

In [ ]:
from detroit_geos import get_detroit_census_geos
from util_detroit import point_to_geo_id
import geopandas as gpd
from features.out_of_state_rental_ownership import OutOfStateRentalOwnership

In [ ]:
r = OutOfStateRentalOwnership(decennial_census_year=2010)
f = r.construct_feature(target_geo_grain='tract')

# Income

In [ ]:
from features.feature_constructor import Feature
from features.income import Income
from detroit_geos import get_detroit_census_geos
from features.feature_constructor import GEO_GRAIN_LEN_MAP

In [ ]:
ftr = Income()
ftr.load_data()
ftr.cleanse_data()

In [ ]:
f = ftr.construct_feature('block')

In [ ]:
df.plot('per_capita_income',legend=True)

# Bus Stops

In [ ]:
from features.smart_bus_stops import smartbusstops
from features.ddot_bus_stops import ddotbusstops
from detroit_geos import get_detroit_census_geos
from features.feature_constructor import GEO_GRAIN_LEN_MAP

In [ ]:
smart = smartbusstops()
ddot = ddotbusstops()

In [ ]:
smart.load_data()
ddot.load_data()
smart.cleanse_data()
ddot.cleanse_data()

In [ ]:
smart_block = smart.construct_feature('block')
ddot_block = ddot.construct_feature('block')

# Greenlight

In [ ]:
from features.project_green_light_locations import projectgreenlightlocations
from detroit_geos import get_detroit_census_geos
from features.feature_constructor import GEO_GRAIN_LEN_MAP

In [ ]:
green = projectgreenlightlocations()

In [ ]:
green.load_data()
green.cleanse_data()

In [ ]:
green_block = green.construct_feature('block')

In [ ]:
green.data.head()

# Rentals

In [1]:
from features.rental_statuses import rentalstatuses
from detroit_geos import get_detroit_census_geos
from features.feature_constructor import GEO_GRAIN_LEN_MAP

In [6]:
rentals = rentalstatuses()
rentals.load_data()
rentals.cleanse_data()
rentals = rentals.construct_feature('block')

Loaded 15,884 rows of data
clean data has 14550 rows
cleansed data validator: geo_id looks good
Generate index not run, or was run on the wrong grain. Creating index on block grain


In [7]:
rentals

block
2.616354e+14    2.0
2.616354e+14    1.0
2.616354e+14    3.0
2.616354e+14    4.0
2.616354e+14    1.0
               ... 
2.616353e+14    2.0
2.616353e+14    NaN
2.616398e+14    NaN
2.616398e+14    NaN
2.616398e+14    NaN
Name: oid, Length: 14691, dtype: float64

# DFD Fire Stations

In [8]:
from features.dfd_fire_stations import dfdfirestations
from detroit_geos import get_detroit_census_geos
from features.feature_constructor import GEO_GRAIN_LEN_MAP

In [9]:
fire_stations = dfdfirestations()
fire_stations.load_data()
fire_stations.cleanse_data()
fire_stations = fire_stations.construct_feature('block')

FileNotFoundError: [Errno 2] No such file or directory: './open_data/DFD_Fire_Station_Locations.csv'

In [10]:
fire_stations.open_data_url()

In [10]:
np.unique(fire_stations)

array([ 1., nan])